<a href="https://colab.research.google.com/github/rushiai/Advance-House-Price/blob/main/google_trend_link%2B_summary_%2B_gpu_master_dic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install newspaper3k
!pip install kora -q
from kora.selenium import wd
import nltk
nltk.download('punkt')

     |████████████████████████████████| 2.3MB 24.0MB/s 
     |████████████████████████████████| 901kB 49.2MB/s 
     |████████████████████████████████| 3.3MB 50.4MB/s 
     |████████████████████████████████| 215kB 25.9MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
     |████████████████████████████████| 7.4MB 47.0MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=6d9b607ecd66c2232ea6e7867ec473245a89f44196fe01e992e0306a1b775997
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398406 sha256=bfbaaf206d27e14ffded96220099eac35ed71e51e610d764bec7b16fe7ac1334
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl 

True

In [2]:
import time
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import glob
import shutil
import numpy as np
import re
from newspaper import Article

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from transformers import pipeline

In [4]:
summarizer = pipeline("summarization", device=0)
# summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)

In [ ]:
categories = ['b','e','m','t','s']
countries = ['US','AU','CA','NZ','GB']
# master_dic = {}

In [6]:
categories = ['b','e']
countries = ['US','CA']
# master_dic = {}

In [ ]:
for count in countries:
    master_dic[count] = {}
    for cat in categories:
        df = pd.DataFrame()
        dfa = pd.DataFrame()
        link = 'https://trends.google.com/trends/trendingsearches/realtime?geo={0}&category={1}'.format(count,cat)
        master_dic[count][cat] = {}
        wd.get(link)
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        page = wd.page_source
        soup = BeautifulSoup(page, 'html.parser')
        result = soup.find("article")
        head_results = soup.find_all("div", class_="summary-text")
        regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        for i in head_results:
            # print(i)
            for j in i:
                # print(j)
                linkss = re.findall(regex,str(j))
                if len(linkss) != 0:
                    # print(re.findall(regex,str(linkss)))
                    z = re.findall(regex,str(linkss))
                    k = pd.Series(z)
                    df = df.append(k,ignore_index=True)

        for i in range(0,len(df)):
            a = pd.Series(df[0][i][:-2])
            dfa = dfa.append(a,ignore_index=True)
        dfa['text'] = np.nan
        for i in range(0,len(df)):
            url = dfa[0][i]
            article = Article(url)
            try:
                article.download()
                article.parse()
                article.nlp()
                art_text = article.text
                s = pd.Series(art_text)
                ****************************************
                article.keywords        image download
                **********************
                # dfa['text'][i] = dfa.append(s,ignore_index=True)
                dfa['text'][i] = art_text
            except:
                print('error ',i)
        dfa = dfa.dropna()
        dfa =dfa.reset_index(inplace=True,drop=True)
        master_dic[count][cat] = dfa

In [7]:
import pickle
# a_file = open("master_23may.pkl", "wb")
# pickle.dump(master_dic, a_file)
# a_file.close()

a_file = open("/content/drive/MyDrive/keep/master_23may.pkl", "rb")
master_dic = pickle.load(a_file)
print(master_dic)

{'US': {'b':                                                     0                                               text
0   https://www.influencive.com/green-electric-sco...  The morning commute is a thing of strife for m...
1   https://www.kpbs.org/news/2021/may/22/californ...  California extended its slow but steady econom...
2   https://ksusentinel.com/2021/05/21/brushless-a...  The market research report 2021 on global Brus...
3   https://fortune.com/2021/05/22/spacs-ipo-stock...  What happens when you mix doses of the Pfizer ...
4   https://www.newswinters.com/uncategorized/1949...  Detailed research added by Ample Market Resear...
5   https://jumbonews.co.uk/uncategorised/3509530/...  A new research study from JCMR with title Glob...
6   https://techstory.in/vitalik-buterin-knew-that...  The recent crash has led many to believe that ...
7   https://www.benzinga.com/money/best-commercial...  Benzinga Money is a reader-supported publicati...
9   https://www.explica.co/cryptocurrency-

In [18]:
for count in countries:
    for cat in categories:
        master_dic[count][cat]['summary'] = np.nan
        for i in range(0,len(master_dic[count][cat]['text'])):
            try:
                if len(master_dic[count][cat]['text'][i]) > 300:
                    try:
                        aaa = summarizer(master_dic[count][cat]['text'][i][:2000], max_length=400, min_length=100, do_sample=False)
                        master_dic[count][cat]['summary'][i] = aaa[0]['summary_text']
                    except:
                        print('*********** {0}****{1}'.format(count,cat))
            except: print('*****************************************************')
        master_dic[count][cat].dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
Your max_length is set to 400, but you input_length is only 394. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


*****************************************************


Your max_length is set to 400, but you input_length is only 361. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 400, but you input_length is only 365. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


*****************************************************
*****************************************************


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
Your max_length is set to 400, but you input_length is only 258. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 400, but you input_length is only 224. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set t

In [19]:
master_dic['US']['b']['summary']

0      Irish company offering €1000 a month for comm...
1      Since February, California has added 390,300 ...
2      The worldwide market for Brushless AC Servo M...
4      Ample Market Research offers a comprehensive ...
5      The year-over-year growth for 2020 is estimat...
6      Vitalik Buterin sold about 13k of his Etherum...
7      Commercial insurance is crucial to protect yo...
9      With India expecting to host 5.2 million prog...
10     Plastic compounding involves an elaborate pro...
11     China's technology giant Huawei has reshuffle...
12     The global Enterprise Resource Planning (ERP)...
Name: summary, dtype: object

In [ ]:
text = master_dic['AU']['b']['text'][8]

In [ ]:
len(master_dic['AU']['b']['text'][8])

5904

In [ ]:
master_dic['AU']['b']['text'][8]

'Get Sample Report Buy Complete Report\n\nAgricultural Biological Control Agent Market research is an intelligence report with meticulous efforts undertaken to study the right and valuable information. The data which has been looked upon is done considering both, the existing top players and the upcoming competitors. Business strategies of the key players and the new entering market industries are studied in detail. Well explained SWOT analysis, revenue share and contact information are shared in this report analysis.\n\n“Agricultural Biological Control Agent Market is growing at a High CAGR during the forecast period 2021-2027. The increasing interest of the individuals in this industry is that the major reason for the expansion of this market”.\n\nGet the PDF Sample Copy (Including FULL TOC, Graphs and Tables) of this report @:\n\nhttps://www.a2zmarketresearch.com/sample?reportId=392072\n\nTop Key Players Profiled in this report are:\n\nBASF SE, Valent BioSciences Corporation, Certis

In [25]:
!apt update &> /dev/null
!apt install imagemagick &> /dev/null
!apt install ffmpeg &> /dev/null
!pip3 install moviepy[optional] &> /dev/null
!sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml

In [26]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/tensorflow/examples/__init__.py
    /usr/local/lib/python3.7/dist-packages/tensorflow/examples/saved_model/__init__.py
    /usr/local/lib/python3.7/dist-packages/tensorflow/examples/saved_model/integration_tests/__init__.py
    /usr/local/lib/python3.7/dist-packages/tensorflow/examples/saved_model/integration_tests/mnist_util.py
    /usr/local/lib/python3.7/dist-packages/tensorflow/include/external/astor_archive/LICENSE
    /usr/local/lib/python3.7/di

In [27]:
!pip install --upgrade "tensorflow==1.15"

y
     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 3.8MB 30.3MB/s 
     |████████████████████████████████| 512kB 56.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=52ffbf14fd09cbfec8f3d4ff6717b0973e2124a87ca385ee798dda156e5ef61b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow-gpu 2.3.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow-gpu 2.3.1 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow-gpu 2.3.1 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
  Fo

In [28]:
import os
!git clone https://github.com/TensorSpeech/TensorFlowTTS
os.chdir("TensorFlowTTS")
!pip install  .
os.chdir("..")
import sys
sys.path.append("TensorFlowTTS/")
print("Downloading Tacotron2 model...")
!gdown --id {"12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW"} -O tacotron2-120k.h5
!gdown --id {"1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY"} -O tacotron2_config.yml
print("Downloading MelGAN-STFT model...")
!gdown --id {"1WB5iQbk9qB-Y-wO8BU6S2TnRiu4VU5ys"} -O melgan.stft-2M.h5
!gdown --id {"1OqdrcHJvtXwNasEZP7KXZwtGUDXMKNkg"} -O melgan.stft_config.yml
print("Downloading MelGAN model...")
!gdown --id {"1A3zJwzlXEpu_jHeatlMdyPGjn1V7-9iG"} -O melgan-1M6.h5
!gdown --id {"1Ys-twSd3m2uqhJOEiobNox6RNQf4txZs"} -O melgan_config.yml
import tensorflow as tf

import yaml
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import AutoProcessor

tacotron2_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/tacotron2/conf/tacotron2.v1.yaml')
tacotron2 = TFAutoModel.from_pretrained(
    config=tacotron2_config,
    pretrained_path="tacotron2-120k.h5",
    name="tacotron2"
)

melgan_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/melgan/conf/melgan.v1.yaml')
melgan = TFAutoModel.from_pretrained(
    config=melgan_config,
    pretrained_path="melgan-1M6.h5",
    name="melgan"
)

melgan_stft_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/melgan_stft/conf/melgan_stft.v1.yaml')
melgan_stft = TFAutoModel.from_pretrained(
    config=melgan_stft_config,
    pretrained_path="melgan.stft-2M.h5",
    name="melgan_stft"
)

print("Downloading ljspeech_mapper.json ...")
!gdown --id {"1YBaDdMlhTXxsKrH7mZwDu-2aODq5fr5e"} -O ljspeech_mapper.json

processor = AutoProcessor.from_pretrained(pretrained_path="./ljspeech_mapper.json")

def do_synthesis(input_text, text2mel_model, vocoder_model, text2mel_name, vocoder_name):
  input_ids = processor.text_to_sequence(input_text)

  # text2mel part
  if text2mel_name == "TACOTRON":
    _, mel_outputs, stop_token_prediction, alignment_history = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        tf.convert_to_tensor([len(input_ids)], tf.int32),
        tf.convert_to_tensor([0], dtype=tf.int32)
    )
  elif text2mel_name == "FASTSPEECH":
    mel_before, mel_outputs, duration_outputs = text2mel_model.inference(
        input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  elif text2mel_name == "FASTSPEECH2":
    mel_before, mel_outputs, duration_outputs, _, _ = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  else:
    raise ValueError("Only TACOTRON, FASTSPEECH, FASTSPEECH2 are supported on text2mel_name")

  # vocoder part
  if vocoder_name == "MELGAN" or vocoder_name == "MELGAN-STFT":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  elif vocoder_name == "MB-MELGAN":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  else:
    raise ValueError("Only MELGAN, MELGAN-STFT and MB_MELGAN are supported on vocoder_name")

  if text2mel_name == "TACOTRON":
    return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()
  else:
    return mel_outputs.numpy(), audio.numpy()

def visualize_attention(alignment_history):
  import matplotlib.pyplot as plt

  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111)
  ax.set_title(f'Alignment steps')
  im = ax.imshow(
      alignment_history,
      aspect='auto',
      origin='lower',
      interpolation='none')
  fig.colorbar(im, ax=ax)
  xlabel = 'Decoder timestep'
  plt.xlabel(xlabel)
  plt.ylabel('Encoder timestep')
  plt.tight_layout()
#   plt.show()
  plt.close()

def visualize_mel_spectrogram(mels):
  mels = tf.reshape(mels, [-1, 80]).numpy()
  fig = plt.figure(figsize=(10, 8))
  ax1 = fig.add_subplot(311)
  ax1.set_title(f'Predicted Mel-after-Spectrogram')
  im = ax1.imshow(np.rot90(mels), aspect='auto', interpolation='none')
  fig.colorbar(mappable=im, shrink=0.65, orientation='horizontal', ax=ax1)
#   plt.show()
  plt.close()

fatal: destination path 'TensorFlowTTS' already exists and is not an empty directory.
Processing /content/TensorFlowTTS
     |████████████████████████████████| 10.3MB 27.6MB/s 
  Using cached https://files.pythonhosted.org/packages/e9/ed/5853ec0ae380cba4588eab1524e18ece1583b65f7ae0e97321f5ff9dfd60/tensorflow_estimator-2.3.0-py2.py3-none-any.whl
     |████████████████████████████████| 6.0MB 50.2MB/s 
     |████████████████████████████████| 4.9MB 39.7MB/s 
  Created wheel for TensorFlowTTS: filename=TensorFlowTTS-0.0-cp37-none-any.whl size=126062 sha256=64557a98f891d6b07063a69651e3819e7ce2d560798bf3e0bbe70ba307afee66
  Stored in directory: /tmp/pip-ephem-wheel-cache-55j5try9/wheels/53/fa/a5/6da6cd434ed077f9027135e4143d7acc9e07a34e6aaebcf476
Successfully built TensorFlowTTS
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.5.0 which is i

Downloading...
From: https://drive.google.com/uc?id=12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW
To: /content/tacotron2-120k.h5
128MB [00:01, 126MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY
To: /content/tacotron2_config.yml
100% 1.49k/1.49k [00:00<00:00, 2.47MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WB5iQbk9qB-Y-wO8BU6S2TnRiu4VU5ys
To: /content/melgan.stft-2M.h5
17.1MB [00:00, 79.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OqdrcHJvtXwNasEZP7KXZwtGUDXMKNkg
To: /content/melgan.stft_config.yml
100% 1.77k/1.77k [00:00<00:00, 1.58MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A3zJwzlXEpu_jHeatlMdyPGjn1V7-9iG
To: /content/melgan-1M6.h5
17.1MB [00:00, 65.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ys-twSd3m2uqhJOEiobNox6RNQf4txZs
To: /content/melgan_config.yml
100% 1.29k/1.29k [00:00<00:00, 1.15MB/s]


ModuleNotFoundError: ignored

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from kora.selenium import wd
import os
import scipy
from scipy.io.wavfile import write
from bing_image_downloader import downloader
import glob
import shutil
import textwrap
# from resizeimage import resizeimage
from moviepy.editor import * # ImageClip
from PIL import Image
import os
from path import Path
import numpy as np
import cv2